In [1]:
import os
import pandas as pd
import h2o
from h2o.automl import H2OAutoML

h2o.init(max_mem_size='12G')
print(h2o.__version__)

Checking whether there is an H2O instance running at http://localhost:54321..... not found.
Attempting to start a local H2O server...
  Java Version: openjdk version "23.0.1" 2024-10-15; OpenJDK Runtime Environment Homebrew (build 23.0.1); OpenJDK 64-Bit Server VM Homebrew (build 23.0.1, mixed mode, sharing)
  Starting server from /Users/ralf/Projects/learning-projects/datascience-bootcamp/sep24_bds_int_medical/.venv/lib/python3.12/site-packages/h2o/backend/bin/h2o.jar
  Ice root: /var/folders/g7/z7m625rj50d5ptztzplh4wgc0000gn/T/tmptqqs16fq
  JVM stdout: /var/folders/g7/z7m625rj50d5ptztzplh4wgc0000gn/T/tmptqqs16fq/h2o_ralf_started_from_python.out
  JVM stderr: /var/folders/g7/z7m625rj50d5ptztzplh4wgc0000gn/T/tmptqqs16fq/h2o_ralf_started_from_python.err
  Server is running at http://127.0.0.1:54321
Connecting to H2O server at http://127.0.0.1:54321 ... successful.


H2O_cluster_uptime:,03 secs
H2O_cluster_timezone:,Europe/Berlin
H2O_data_parsing_timezone:,UTC
H2O_cluster_version:,3.46.0.6
H2O_cluster_version_age:,12 days
H2O_cluster_name:,H2O_from_python_ralf_3yx0wr
H2O_cluster_total_nodes:,1
H2O_cluster_free_memory:,11.98 Gb
H2O_cluster_total_cores:,8
H2O_cluster_allowed_cores:,8
H2O_cluster_status:,"locked, healthy"


3.46.0.6


In [2]:
%%time

import warnings
warnings.filterwarnings('ignore')

CPU times: user 20 μs, sys: 3 μs, total: 23 μs
Wall time: 24.1 μs


In [3]:
%%time

train_data_file = os.path.join('..', '..', '..', '..', 'data', 'raw', 'train.csv')
train_data = pd.read_csv(train_data_file, index_col=0, low_memory=False)
extra_train_data_file = os.path.join('..', '..', '..', '..', 'data', 'interim', 'all_test_2h.csv')
extra_train_data = pd.read_csv(extra_train_data_file, low_memory=False)

all_train_data = pd.concat([train_data, extra_train_data], axis=0)
all_train_data.head()

CPU times: user 12.5 s, sys: 1.92 s, total: 14.4 s
Wall time: 16.3 s


,p_num,time,bg-5:55,bg-5:50,bg-5:45,bg-5:40,bg-5:35,bg-5:30,bg-5:25,bg-5:20,...,activity-0:40,activity-0:35,activity-0:30,activity-0:25,activity-0:20,activity-0:15,activity-0:10,activity-0:05,activity-0:00,bg+1:00
p01_0,p01,06:10:00,NaN,NaN,9.6,NaN,NaN,9.7,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,13.4
p01_1,p01,06:25:00,NaN,NaN,9.7,NaN,NaN,9.2,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,12.8
p01_2,p01,06:40:00,NaN,NaN,9.2,NaN,NaN,8.7,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,15.5
p01_3,p01,06:55:00,NaN,NaN,8.7,NaN,NaN,8.4,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,14.8
p01_4,p01,07:10:00,NaN,NaN,8.4,NaN,NaN,8.1,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,12.7


In [4]:
from pipelines import preprocessing_pipeline

all_train_data = preprocessing_pipeline.fit_transform(all_train_data)
all_train_data.head()

,bg-3:00,bg-2:55,bg-2:50,bg-2:45,bg-2:40,bg-2:35,bg-2:30,bg-2:25,bg-2:20,bg-2:15,...,cals-0:35,cals-0:30,cals-0:25,cals-0:20,cals-0:15,cals-0:10,cals-0:05,cals-0:00,bg+1:00,hour
p01_0,13.9,14.000000,14.100000,14.2,14.200000,14.200000,14.2,14.600000,15.000000,15.4,...,6.0,6.0,6.0,6.0,6.0,6.0,6.0,6.0,13.4,06
p01_1,14.2,14.200000,14.200000,14.2,14.600000,15.000000,15.4,16.000000,16.600000,17.2,...,6.0,6.0,6.0,6.0,6.0,6.0,6.0,6.0,12.8,06
p01_2,14.2,14.600000,15.000000,15.4,16.000000,16.600000,17.2,17.533333,17.866667,18.2,...,6.0,6.0,6.0,6.0,6.0,6.0,6.0,6.0,15.5,06
p01_3,15.4,16.000000,16.600000,17.2,17.533333,17.866667,18.2,18.266667,18.333333,18.4,...,6.0,6.0,6.0,6.0,6.0,6.0,6.0,6.0,14.8,06
p01_4,17.2,17.533333,17.866667,18.2,18.266667,18.333333,18.4,18.266667,18.133333,18.0,...,6.0,6.0,6.0,6.0,6.0,6.0,6.0,6.0,12.7,07


In [5]:
%%time

train_h2o = h2o.H2OFrame(all_train_data)
train_h2o.head()

y = 'bg+1:00'
x = [col for col in train_h2o.columns if col != y]
print(x)
print(y)

Parse progress: |████████████████████████████████████████████████████████████████| (done) 100%
['bg-3:00', 'bg-2:55', 'bg-2:50', 'bg-2:45', 'bg-2:40', 'bg-2:35', 'bg-2:30', 'bg-2:25', 'bg-2:20', 'bg-2:15', 'bg-2:10', 'bg-2:05', 'bg-2:00', 'bg-1:55', 'bg-1:50', 'bg-1:45', 'bg-1:40', 'bg-1:35', 'bg-1:30', 'bg-1:25', 'bg-1:20', 'bg-1:15', 'bg-1:10', 'bg-1:05', 'bg-1:00', 'bg-0:55', 'bg-0:50', 'bg-0:45', 'bg-0:40', 'bg-0:35', 'bg-0:30', 'bg-0:25', 'bg-0:20', 'bg-0:15', 'bg-0:10', 'bg-0:05', 'bg-0:00', 'insulin-3:00', 'insulin-2:55', 'insulin-2:50', 'insulin-2:45', 'insulin-2:40', 'insulin-2:35', 'insulin-2:30', 'insulin-2:25', 'insulin-2:20', 'insulin-2:15', 'insulin-2:10', 'insulin-2:05', 'insulin-2:00', 'insulin-1:55', 'insulin-1:50', 'insulin-1:45', 'insulin-1:40', 'insulin-1:35', 'insulin-1:30', 'insulin-1:25', 'insulin-1:20', 'insulin-1:15', 'insulin-1:10', 'insulin-1:05', 'insulin-1:00', 'insulin-0:55', 'insulin-0:50', 'insulin-0:45', 'insulin-0:40', 'insulin-0:35', 'insulin-0:30', '

In [ ]:
%%time

aml = H2OAutoML(max_models=500, seed=1000, max_runtime_secs=3600)
aml.train(x=x, y=y, training_frame=train_h2o)

AutoML progress: |
14:25:16.699: AutoML: XGBoost is not available; skipping it.

█

In [ ]:
%%time

# View the AutoML Leaderboard
lb = aml.leaderboard
lb.head(rows=lb.nrows)

In [ ]:
%%time
test_file = os.path.join('..', '..', '..', '..', 'data', 'raw', 'test.csv')
test_data = h2o.import_file(test_file)

y_pred = aml.predict(test_data=test_data)

In [ ]:
import pandas as pd

sample_submission = pd.read_csv(os.path.join('..', '..', '..', '..', 'data', 'raw', 'sample_submission.csv'), index_col=0)

sample_submission['bg+1:00'] = y_pred.as_data_frame().values
sample_submission.to_csv(f'submission-{os.path.basename(os.getcwd())}.csv')


In [ ]:
from sklearn.metrics import root_mean_squared_error, r2_score, PredictionErrorDisplay
import matplotlib.pyplot as plt

y_train = train_h2o[y].as_data_frame().values
y_pred = aml.leader.predict(train_h2o).as_data_frame().values

print(f'RMSE: {root_mean_squared_error(y_true=y_train, y_pred=y_pred)}')
print(f'R2: {r2_score(y_true=y_train, y_pred=y_pred)}')

fig, axs = plt.subplots(ncols=2, figsize=(8, 4))
PredictionErrorDisplay.from_predictions(
    y_true=y_train,
    y_pred=y_pred,
    kind="actual_vs_predicted",
    subsample=100,
    ax=axs[0],
    random_state=0,
)
axs[0].set_title("Actual vs. Predicted values")
PredictionErrorDisplay.from_predictions(
    y_true=y_train,
    y_pred=y_pred,
    kind="residual_vs_predicted",
    subsample=100,
    ax=axs[1],
    random_state=0,
)
axs[1].set_title("Residuals vs. Predicted Values")
fig.suptitle("Plotting cross-validated predictions")
plt.tight_layout()
plt.show();